# Notebook for Testing LoRA Module

It retrieves the gradient of each layer to verify the LoRA implementation.
The code is written to build and train a two-layer network.

In [1]:
import torch
import torch.nn as nn

In [2]:
input_dim = 5
unit_dim = 10
rank = 3
alpha = 2
scaling = alpha / rank

In [3]:
x = torch.ones(input_dim)

## Layer declaration

In [4]:
# Layer 1
W1 = torch.tensor([[0.06174474955,0.1174401641,0.2721953988,0.4354655147,0.1299865246,0.4527693391,0.05677324533,0.4392425418,-0.09656989574,0.1562970877],
                  [0.1845430732, -0.1462468505, -0.07894665003, -0.2561007142, 0.4955579638, -0.5607186556, 0.5864921212, -0.2875695825, -0.1474361122, -0.02825164795 ],
                  [0.3690062165, 0.3948656917, 0.03654950857, -0.02532708645, 0.08607029915, -0.1356176734, 0.5383419394, 0.4251096845, -0.5426012278, -0.2056793869 ],
                  [-0.5222446918, 0.1874243021, -0.6068810225, -0.1666627228, 0.4207344651, 0.5782606006, 0.3518435359, -0.4549242854, 0.4680324197, 0.4681274295 ],
                  [0.6054095626, -0.03338348866, 0.3784089684, 0.3806253076, -0.04872518778, 0.02590221167, 0.3548445106, 0.2262667418, -0.4828489125, 0.279080689]], requires_grad=False)
l1_lora_A = torch.tensor([[0.2423502803,0.1420624852,-0.6177302003],
                       [0.06473237276,0.7701891065,0.4479354024],
                       [0.03784239292,-0.6825880408,-0.1478098631],
                       [-0.04572540522,-0.4078016579,-0.5432882905],
                       [0.474986732,0.4103543162,-0.0759498477 ]], requires_grad=True)
l1_lora_B = torch.tensor([[-0.3851322532,0.09298342466,-0.4956409037,-0.6538358927,-0.2389453351,0.1598351598,-0.4759981334,0.1523081064,-0.3772909343,0.1588820219],
                       [-0.1542311311,0.6029348373,0.547023654 ,0.2470450997, -0.06800460815, -0.1908912063, 0.1536229849, -0.08555674553, 0.5466840267, 0.268527925], 
                       [-0.544470787, -0.5975359082, 0.6383447647, 0.2265913486, 0.2080762982, 0.2318511605, -0.4471458197, -0.3935126066, -0.1927332282, -0.5041899085]], requires_grad=True)

In [5]:
# Layer 2
"""
W2 = torch.tensor([[0.2746129632,-0.2021881342,0.1181225777,-0.1492973864,-0.1916511655,0.07689666748,-0.5056296587,-0.06725865602,0.1470898986,0.5349867344],
[0.5027537346,-0.4359380901,0.1673734188,-0.3189095259,0.1479495764,-0.37101686,0.5425735712,0.1677217484,0.08966255188,-0.270255506],
[-0.09380450845,-0.03690469265,-0.02771747112,-0.2799677551,0.1352985501,-0.3735800982,-0.1774537563,-0.4268126786,0.191431582,0.1712504625],
[-0.2002454698,-0.3963507414,0.3049122095,-0.3323773742,0.4924803972,-0.1438043714,0.1780392528,0.3516305089,-0.5328555703,-0.4413534701],
[0.134571135,0.3702001572,0.1902346015,-0.4424520433,0.5169898272,0.5219352245,0.4142902493,-0.03434085846,0.01054298878,0.5222656727],
[-0.4866901934,0.1148525476,-0.05350241065,0.2621001601,-0.5258271694,-0.5047944784,-0.06385248899,-0.2379230559,0.5253609419,-0.4160538316],
[-0.1539708674,-0.2233172357,-0.02093011141,-0.417662859,0.2066679597,-0.1993894577,0.4167904854,-0.09392020106,0.4581540823,-0.4774525166],
[-0.3102058172,0.2108426094,0.07141083479,0.07295823097,0.3999498487,-0.2570029199,0.009824991226,0.02546697855,0.4564971924,-0.4448159039],
[0.4613550901,0.0831952095,-0.456677407,0.4702703953,-0.2434971035,-0.1987477839,-0.5374728441,0.1833888888,0.3750169873,-0.4033452272],
[1.1612212062,0.236974597,0.3739697933,-0.2306940854,-0.257725209,-0.3470464945,-0.111931771,0.0947701335,0.05786299706,-0.5256958604]], requires_grad=False)
l2_lora_A = torch.tensor([[-0.4552562833,0.4469414949,-0.1768959761],
[-0.6729863286,-0.4803910553,0.2416050434],
[0.09459280968,-0.3124976456,0.2768244147],
[0.319565773,-0.2874039412,0.6279906034],
[-0.09064364433,-0.3413772583,0.3479804993],
[0.1034774184,-0.1411746144,0.1250603199],
[0.5381102562,0.09817099571,0.1887565851],
[-0.3762579858,0.5320177078,0.6151647568],
[0.2446473241,-0.07184249163,-0.0690267086],
[0.4706766605,0.6502498388,0.2710389495]], requires_grad=True)
l2_lora_B = torch.tensor([[-0.5214788914,-0.2752289772,0.3628137112,0.4263672829,-0.1198128462,-0.1406210065,0.2383747697,0.5178172588,-0.3399599195,0.1104281545],
[-0.2537863255,0.518676281,0.6323760748,0.2615989447,0.1202003956,0.3060603142,0.2169466019,0.001799464226,0.04511839151,0.6196956635],
[-0.3661335409,0.1956441998,-0.1428444982,-0.1034606099,0.1614290476,0.1445599198,-0.03414833546,-0.653287828,-0.04058235884,-0.2696067393]], requires_grad=True)
"""
W2 = torch.tensor([[0.3170956969, -0.2334667146, 0.13639616970000001, -0.17239376899999997, -0.2212997079], [0.08879262209, -0.583850801, -0.07766360044, 0.1698447466, 0.617749393], [0.5805299878, -0.5033779144, 0.1932661533, -0.3682450056, 0.17083746190000001], [-0.4284133315, 0.6265099645000001, 0.1936683655, 0.10353338720000001, -0.3120641708], [-0.1083161235, -0.04261386395, -0.032005429269999996, -0.323278904, 0.1562293172], [-0.43137314920000003, -0.20490592719999998, -0.49284079670000003, 0.2210460901, 0.1977429986], [-0.2312235534, -0.4576663971, 0.3520822525, -0.3837963343, 0.5686674714], [-0.16605097060000001, 0.2055820227, 0.40602785350000004, -0.6152886152, -0.5096310377000001], [0.15538936849999999, 0.4274702668, 0.2196639776, -0.5108996034000001, 0.596968472], [0.6026788354, 0.47838109729999995, -0.03965342045, 0.012173950670000001, 0.6030604243]], requires_grad=False)
l2_lora_A = torch.tensor([[-0.603664875, 0.1424570084, -0.06636154652000001], [0.32509517670000004, -0.6522083282, -0.6261205077], [-0.07919925451, -0.2951072156, 0.651630044], [-0.5160512328, -0.1909773648, -0.2769909501], [-0.02596056461, -0.5180470347, 0.2563400269], [-0.24731215829999997, 0.5169649124, -0.1164936423], [0.5682700871999999, -0.592206955, -0.3847629726], [0.2615180612, 0.08857423067, 0.09049355983999999], [0.4960767031, -0.318772912, 0.01218640804], [0.03158789873, 0.5662150383, -0.551726222]], requires_grad=True)
l2_lora_B = torch.tensor([[0.7294663787000001, 0.1315432191, -0.7220703363, 0.7435627579999999, -0.3850027025], [-0.314247787, -0.8498191237, 0.289963305, 0.5929538608, -0.6377447843999999], [0.2549130321, 0.3746896386, 0.5912981629, -0.364759326, -0.40749931340000006]], requires_grad=True)

## Forward

In [6]:
# forward layer 1
tmp = nn.functional.linear(x, l1_lora_A.T)
out = nn.functional.linear(tmp, l1_lora_B.T)
hidden = nn.functional.linear(x, W1.t())
layer1_out = hidden + out * scaling

# add a hook to monitor gradient of layer1_ouut
layer1_out.register_hook(lambda gOut: print("layer1 gOut: \n", gOut))

# print forward layer 1 output
print(x)
print(tmp)
print(out)
print(hidden)
print(layer1_out)

tensor([1., 1., 1., 1., 1.])
tensor([ 0.7742,  0.2322, -0.9368], grad_fn=<SqueezeBackward4>)
tensor([ 0.1761,  0.7718, -0.8547, -0.6611, -0.3957, -0.1378,  0.0861,  0.4667,
         0.0154,  0.6577], grad_fn=<SqueezeBackward4>)
tensor([ 6.9846e-01,  5.2010e-01,  1.3262e-03,  3.6800e-01,  1.0836e+00,
         3.6060e-01,  1.8883e+00,  3.4813e-01, -8.0142e-01,  6.6957e-01])
tensor([ 0.8159,  1.0346, -0.5685, -0.0727,  0.8198,  0.2687,  1.9457,  0.6593,
        -0.7911,  1.1080], grad_fn=<AddBackward0>)


In [7]:
# forward layer 2
tmp2 = nn.functional.linear(layer1_out, l2_lora_A.T)
out2 = nn.functional.linear(tmp2, l2_lora_B.T)
hidden2 = nn.functional.linear(layer1_out, W2.t())
layer2_out = hidden2 + out2 * scaling

# add a hook to monitor gradient of layer2_out and tmp2
tmp2.register_hook(lambda gTmp: print("layer2: gTmp: \n", gTmp))
layer2_out.register_hook(lambda gOut: print("layer2: gOut: \n", gOut))

# print forward layer 2 output
print(layer1_out)
print(tmp2)
print(out2)
print(hidden2)
print(layer2_out)

tensor([ 0.8159,  1.0346, -0.5685, -0.0727,  0.8198,  0.2687,  1.9457,  0.6593,
        -0.7911,  1.1080], grad_fn=<AddBackward0>)
tensor([ 0.7593, -0.8769, -2.1833], grad_fn=<SqueezeBackward4>)
tensor([ 0.2729,  0.0271, -2.0935,  0.8410,  1.1567],
       grad_fn=<SqueezeBackward4>)
tensor([-0.1675, -1.2070,  0.4833, -0.7034,  1.5318],
       grad_fn=<SqueezeBackward4>)
tensor([ 0.0144, -1.1890, -0.9124, -0.1428,  2.3029], grad_fn=<AddBackward0>)


## Backpropagation

In [8]:
labels = torch.tensor([ 1.0, 0, 0, 0, 0])
loss = nn.MSELoss()(layer2_out, labels)
loss.retain_grad()
loss.backward(retain_graph=True)

layer2: gOut: 
 tensor([-0.3942, -0.4756, -0.3650, -0.0571,  0.9212])
layer2: gTmp: 
 tensor([-0.3225, -0.1327, -0.5660])
layer1 gOut: 
 tensor([-0.0444,  1.1665, -0.1857, -0.1446,  0.1691,  0.6940,  0.8392, -0.7621,
         0.1098,  0.3311])


In [9]:
# manual computation of gradient
derivative = torch.tensor([-0.3942, -0.4756, -0.3650, -0.0571,  0.9212])
djdlb = tmp2.unsqueeze(1) @ derivative.unsqueeze(1).T * scaling
print(djdlb)

tensor([[-0.1995, -0.2407, -0.1848, -0.0289,  0.4663],
        [ 0.2305,  0.2781,  0.2134,  0.0334, -0.5386],
        [ 0.5738,  0.6923,  0.5313,  0.0831, -1.3409]], grad_fn=<MulBackward0>)


In [10]:
print(tmp2.grad)

None


/tmp/ipykernel_117787/949293141.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /opt/conda/conda-bld/pytorch_1702400430266/work/build/aten/src/ATen/core/TensorBody.h:489.)
  print(tmp2.grad)


In [11]:
print(l2_lora_A.grad)
print(l2_lora_B.grad)

tensor([[-0.2631, -0.1083, -0.4618],
        [-0.3337, -0.1373, -0.5856],
        [ 0.1833,  0.0755,  0.3218],
        [ 0.0235,  0.0097,  0.0412],
        [-0.2644, -0.1088, -0.4640],
        [-0.0867, -0.0357, -0.1521],
        [-0.6275, -0.2583, -1.1013],
        [-0.2126, -0.0875, -0.3732],
        [ 0.2551,  0.1050,  0.4478],
        [-0.3573, -0.1471, -0.6272]])
tensor([[-0.1996, -0.2407, -0.1847, -0.0289,  0.4663],
        [ 0.2305,  0.2780,  0.2134,  0.0334, -0.5385],
        [ 0.5738,  0.6923,  0.5312,  0.0831, -1.3408]])


In [12]:
print(l1_lora_A.grad)
print(l1_lora_B.grad)

tensor([[-0.0811,  0.5146, -0.5943],
        [-0.0811,  0.5146, -0.5943],
        [-0.0811,  0.5146, -0.5943],
        [-0.0811,  0.5146, -0.5943],
        [-0.0811,  0.5146, -0.5943]])
tensor([[-0.0229,  0.6021, -0.0959, -0.0746,  0.0873,  0.3582,  0.4331, -0.3933,
          0.0567,  0.1709],
        [-0.0069,  0.1806, -0.0288, -0.0224,  0.0262,  0.1074,  0.1299, -0.1180,
          0.0170,  0.0513],
        [ 0.0278, -0.7285,  0.1160,  0.0903, -0.1056, -0.4334, -0.5241,  0.4760,
         -0.0686, -0.2068]])
